# Credo AI Platform Integration

This notebook is a comprehensive overview of how Credo AI Lens interacts with the Credo AI Platform.

Connecting Lens to the Platform requires that you have already created a Use-Case, and a policy pack defining how the associated models and data should be assessed.

**Find the code**

Download this notebook from [github](https://github.com/credo-ai/credoai_lens/blob/main/docs/notebooks/platform_integration.ipynb).

If you want to just get the boilerplate code, see [example in this script](https://raw.githubusercontent.com/credo-ai/credoai_lens/develop/docs/example_code.py).

## Glossary
A list of useful keywords:  

- **Credo AI Platform**: Also referred to as simply "Platform". The central AI governance/reporting Platform, found at [https://app.credo.ai/](https://app.credo.ai/)

- **credoconfig**: configuration file to be copied in the user's home folder

- **use_case_name**: The name of your Use Case as it is registered on Credo AI Platform

- **policy_pack**: A set of governance controls that a Use Case needs to satisfy. A Use Case can have multiple policy packs applied to it.

- **policy_pack_key**: A unique identifier for a policy pack (where do we get this?)

- **assessment_plan**: A set of evaluations that `Lens` needs to run in order to satisfy a policy pack requirements.

- **assessment_plan_url**: The link to the assessment plan, this is generated in the Platform and used to download the assessment plan in the Governance object. See example below.

- **evidence**: The output of a `Lens` evaluation, formatted specifically to be uploaded to the platform.


## Setup API Connection with the Platform

### Get a config file
This file contains all the necessary information to connect Lens to the Credo AI Platform.  

To generate the config file, once you logged into the platform, click on your name icon (top left) and follow:  

`My Settings -> Tokens -> Plus Sign -> Generate`

Immediately after generating the token, you will be given the possibility to download the config file.

The default location/file name Lens is expecting is `~/.credoconfig`, where `~` is your home folder. You can specify any other location when you are initializing the `Governance` object (see below).

### Get your ML environment ready
 In this tutorial we will emulate the modeling phase by running a quick script. This script loads a dataset, splits it into training and testing, and fits a model. You can see the full script [here](https://github.com/credo-ai/credoai_lens/blob/release/1.0.0/docs/notebooks/training_script.py).

In [1]:
from credoai.governance import Governance
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData
from credoai.datasets import fetch_credit_model

# model and data are defined by this script
X_test, y_test, sensitive_features_test, model = fetch_credit_model()

## Governance in 5 Minutes

In [ ]:
# set up governance
gov = Governance()  # Specify config_path if your config file is not the default one: ~/.credoconfig
url = 'your assessment url'
gov.register(assessment_plan_url=url)

# set up model and data artifacts
credo_model = ClassificationModel(name="credit_default_classifier", model_like=model)

credo_data = TabularData(
    name="UCI-credit-default",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

# Initialization of the Lens object
lens = Lens(model=credo_model, assessment_data=credo_data, governance=gov)

# Run Lens
lens.run()
# Update governance object with newly created evidence
lens.send_to_governance()
# Export to platform
gov.export()

## Breaking Down the Steps

### Create Governance
In this phase we create the Governance object with all the necessary information required to run a battery of assessments defined by your use case and policy pack.

In [ ]:
gov = Governance()

After initializing the `Governance` object, we need to register an assessment plan. This can be achieved in three ways:  

1. Using the `assessment_plan_url`. In the Platform: your_use_case -> Reports -> Your_policy_pack -> **ASSESSMENT PLAN** -> **Copy URL**
2. Using the assessment plan json. Follow steps in 1., but click on **Download** as the last step. (more details in [Air Gap Environment](##Air-Gap-Environment))
3. Using use_case_name and policy pack key as they are defined in the Platform

In the tutorial we will follow 1. 


In [ ]:
url = 'your_assessment_url'
gov.register(assessment_plan_url=url)

# Alternatively method 2.
# gov.register(use_case_name='your_use_case_name', policy_pack_key='your_policy_pack_id')

### Run Lens
This part is very similar to a standard `Lens` run. See the [quickstart notebook](https://credoai-lens.readthedocs.io/en/stable/notebooks/quickstart.html) for more general information on `Lens`.

In [ ]:
credo_model = ClassificationModel("credit_default_classifier", model)

test_data = TabularData(
    name="UCI-credit-default-test",
    X=X_test,
    y=y_test,
    sensitive_features=sensitive_features_test,
)

lens = Lens(
    model=credo_model,
    assessment_data=test_data,
    governance=gov
)

When `Governance` is included in the initialization of Lens, there is no need to specify which evaluators to include in the pipeline. Under the hood, `Lens` converts the assessment plan contained in `gov` into a list of evaluators.

A simple `lens.run()` is sufficient to run all the required assessments.

In [ ]:
lens.run()

If the execution proceeded according to expectations, the evidence, i.e., the results of the assessments, can be sent to the `Governance` object and finally exported to the platform. In the example below, while sending the evidence to governance, we opted to overwrite any prior evidence.

In [ ]:
lens.send_to_governance(overwrite_governance=True)
gov.export()

That's it! With just a few lines of code, a set of evidences documenting compliance to a set of controls defined by a policy pack has been uploaded to the Platform.

## Air Gap Environment

If you cannot interact with Credo AI Platform via the internet, this section is for you!

Instead of Lens automating the communication between your assessment environment and governance, you will instead have to download an assessment plan, provide it to `Governance`, run `Lens` and finally upload the evidence(assessments results) to the platform.



### Getting the assessment plan
The asset that you have to get _from_ the governance app is the assessment plan. This can be found in:   

your_use_case -> Reports -> Your_policy_pack -> **ASSESSMENT PLAN** -> **Download**  

Once you download the assessments plan, you can pass it to the Governance object, see code below.


In [ ]:
gov.register(assessment_plan_file='your_assessment_file.json')

### Exporting Results

Lens can export evidence to file as a json object. This is generally useful if you want to store your results locally, but particularly useful if you are in an air gapped environment where you cannot access Credo AI's Governance Platform directly from your development environment.

Doing this only requires a one line change in our current flow. We change the `export` argument from `Governance` to a local path:


In [ ]:
# To save the evidence locally
gov.export('evidence_test.json')

Running the above will create a folder that you specified, if it doesn't exist. It will then create a json file with the assessment results, which can be uploaded to Credo AI's Governance Platform. This json file can be uploaded by going to:  

your_use_case -> Reports -> Your_policy_pack -> **UPLOAD LENS EVIDENCE**